In [26]:
## animate Electric field

%pylab tk
import numpy as np
from matplotlib import pyplot as plt
import os
from ReadBinary import *

indStart = 0

Ey = GetArrays("../data/2D/E-y.data", indStart=indStart, indEnd=None)[:, 0, :, :]
Ez = GetArrays("../data/2D/E-z.data", indStart=indStart, indEnd=None)[:, 0, :, :]
#Hx = GetArrays("../data/2D/H-x.data", indStart=indStart, indEnd=None)[:, 0, :, :]
Jey = GetArrays("../data/2D/Je-y.data", indStart=indStart, indEnd=None)[:, 0, :, :]
Jez = GetArrays("../data/2D/Je-z.data", indStart=indStart, indEnd=None)[:, 0, :, :]
Wp2y = GetArrays("../data/2D/Wp2-y.data", indStart=0, indEnd=1)[0, 0, :, :]


print("shape Ey: ", Ey.shape)
print("shape Ez: ", Ez.shape)

print("max Jey: ", np.max(np.abs(Jey)))
print("max Jez: ", np.max(np.abs(Jez)))

show_pml = True

Ey_pml_r = None
Ez_pml_r = None
Ey_pml_l = None
Ez_pml_l = None
Ey_pml_t = None
Ez_pml_t = None
if show_pml:
    if os.path.exists("../data/2D/E-pml-r-y.data"):
        Ey_pml_r = GetArrays("../data/2D/E-pml-r-y.data", indStart=0, indEnd=None)[:, 0, :, :]
    if os.path.exists("../data/2D/E-pml-r-z.data"):
        Ez_pml_r = GetArrays("../data/2D/E-pml-r-z.data", indStart=0, indEnd=None)[:, 0, :, :]
    if os.path.exists("../data/2D/E-pml-l-y.data"):
        Ey_pml_l = GetArrays("../data/2D/E-pml-l-y.data", indStart=0, indEnd=None)[:, 0, :, :]
    if os.path.exists("../data/2D/E-pml-l-z.data"):
        Ez_pml_l = GetArrays("../data/2D/E-pml-l-z.data", indStart=0, indEnd=None)[:, 0, :, :]
    ey_nz = Ey.shape[2]
    ez_nz = Ez.shape[2]
    Ey = np.concatenate((Ey_pml_l[:, :,:-1], Ey[:, :,:-1], Ey_pml_r[:, :,:]), axis=-1)
    Ez = np.concatenate((Ez_pml_l[:, :,:], Ez[:, :,:], Ez_pml_r[:, :,:]), axis=-1)

    if os.path.exists("../data/2D/E-pml-t-y.data"):
        Ey_pml_t = GetArrays("../data/2D/E-pml-t-y.data", indStart=0, indEnd=None)[:, 0, :, :]
        pml_t_nt, pml_t_ny, pml_t_nz = Ey_pml_t.shape

        nz_right = int((Ey.shape[2] - ey_nz) / 2)
        Ey_pml_tr = np.zeros((pml_t_nt, pml_t_ny, nz_right))
        if os.path.exists("../data/2D/E-pml-tr-y.data"):
            Ey_pml_tr = GetArrays("../data/2D/E-pml-tr-y.data", indStart=0, indEnd=None)[:, 0, :, :]
            nz_right = Ey_pml_tr.shape[2]
            print("Ey_pml_tr loaded!")

        nz_left = Ey.shape[2] - ey_nz - nz_right + 1
        Ey_pml_tl = np.zeros((pml_t_nt, pml_t_ny, nz_left))

        Ey_pml_t = np.concatenate((Ey_pml_tl, Ey_pml_t[:, :,:-1], Ey_pml_tr), axis=-1)        
        Ey = np.concatenate((Ey[:, :,:], Ey_pml_t[:, :,:]), axis=1)
    if os.path.exists("../data/2D/E-pml-t-z.data"):
        Ez_pml_t = GetArrays("../data/2D/E-pml-t-z.data", indStart=0, indEnd=None)[:, 0, :, :]
        pml_t_nt, pml_t_ny, pml_t_nz = Ez_pml_t.shape

        nz_right = int((Ez.shape[2] - ez_nz) / 2)
        Ez_pml_tr = np.zeros((pml_t_nt, pml_t_ny, nz_right))
        if os.path.exists("../data/2D/E-pml-tr-z.data"):
            Ez_pml_tr = GetArrays("../data/2D/E-pml-tr-z.data", indStart=0, indEnd=None)[:, 0, :, :]
            nz_right = Ez_pml_tr.shape[2]
            print("Ez_pml_tr loaded!")

        nz_left = Ez.shape[2] - ez_nz - nz_right
        Ez_pml_tl = np.zeros((pml_t_nt, pml_t_ny, nz_left))

        Ez_pml_t = np.concatenate((Ez_pml_tl, Ez_pml_t[:, :,:], Ez_pml_tr), axis=-1)        
        Ez = np.concatenate((Ez[:, :-1,:], Ez_pml_t[:, :,:]), axis=1)



Populating the interactive namespace from numpy and matplotlib


/home/nima/.local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shape']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


shape Ey:  (54, 600, 601)
shape Ez:  (54, 601, 600)
max Jey:  0.0
max Jez:  0.0
Ey_pml_tr loaded!
Ez_pml_tr loaded!


In [27]:
import matplotlib.animation as animation

E = np.sqrt(Ey[:, :, :-1]**2 + Ez[:, :-1, :]**2)
#E = np.log10(E + 1.0e-10)
Je = np.sqrt(Jey[:, :, :-1]**2 + Jez[:, :-1, :]**2)
Je = np.log10(Je + 1.0e-10)

shape = E.shape[1:]

animate = True
save_animation = False

if animate:
    def animate_E(n):
        #E_n = np.concatenate((E_pml_l[n, :,:-1], E[n, :,:-1], E_pml_r[n, :,:]), axis=1)
        E_n = E[n, :,:-1]
        plt.clf()
        fig = plt.imshow(np.real(E_n), cmap="rainbow", origin='lower', aspect='auto', interpolation="bicubic")
        plt.colorbar()
        plt.xlabel(r'$z$', fontsize=16)
        plt.ylabel(r'$y$', fontsize=16)
        #plt.title(r'$|E|$ total', fontsize=20)
        plt.title(r'$E_y$', fontsize=20)
        plt.contour(np.real(Wp2y))
        #plt.contour(np.real(Je[n, :, :]))
        plt.pause(0.05)
        
        return fig
    
    if not save_animation:
        plt.ion()
        plt.figure(figsize=(7,6*(shape[0]/shape[1])))

        for n in range(E.shape[0]):
            animate_E(n)
    else:
        fig = plt.figure(figsize=(7,6*(shape[0]/shape[1])))
        anim = animation.FuncAnimation(fig, animate_E, frames=E.shape[0], interval=1, repeat=False)
        anim.save("../data/2D/" + 'Efield-anim.mp4', writer="ffmpeg", fps=15, dpi=200)


/home/nima/.local/lib/python3.6/site-packages/matplotlib/contour.py:1230: UserWarning: No contour levels were found within the data range.
  warnings.warn("No contour levels were found"


In [2]:

plt.figure()
sigEy_pml_t = GetArrays("../data/2D/sigE-pml-tr-y.data", indStart=0, indEnd=None)[0, 0, :, :]
plt.imshow(np.real(sigEy_pml_t))
plt.colorbar()
plt.show()

plt.figure()
sigHy_pml_t = GetArrays("../data/2D/sigH-pml-tr-y.data", indStart=0, indEnd=None)[0, 0, :, :]
plt.imshow(np.real(sigHy_pml_t))
plt.colorbar()
plt.show()


In [25]:
Hx_t = GetArrays("../data/2D/H-pml-t-x.data", indStart=indStart, indEnd=None)[:, 0, :, :]
plt.imshow(np.real(Hx_t[:,:,-1]))
plt.colorbar()

In [53]:
Ez_t = GetArrays("../data/2D/E-pml-t-z.data", indStart=indStart, indEnd=None)[:, 0, :, :]
plt.imshow(np.real(Ez_t[:,:,-1]))
plt.colorbar()

In [50]:
Ey_t = GetArrays("../data/2D/E-pml-t-y.data", indStart=indStart, indEnd=None)[:, 0, :, :]
plt.imshow(np.real(Ey_t[:,:,-1]))
plt.colorbar()

In [29]:
Ez_r = GetArrays("../data/2D/E-pml-r-z.data", indStart=indStart, indEnd=None)[:, 0, :, :]
plt.imshow(np.real(Ez_r[:,-1,:]))
plt.colorbar()

In [32]:
Ey_r = GetArrays("../data/2D/E-pml-r-y.data", indStart=indStart, indEnd=None)[:, 0, :, :]
plt.imshow(np.real(Ey_r[:,-1,100:]))
plt.colorbar()

In [52]:
Ez_tr = GetArrays("../data/2D/E-pml-tr-z.data", indStart=indStart, indEnd=None)[:, 0, :, :]
plt.imshow(np.real(Ez_tr[:,:,0]))
plt.colorbar()

In [49]:
Ey_tr = GetArrays("../data/2D/E-pml-tr-y.data", indStart=indStart, indEnd=None)[:, 0, :, :]
plt.imshow(np.real(Ey_tr[:,:,0]))
plt.colorbar()

In [56]:
plt.imshow(np.real(Ey_tr[:,:,0] - Ey_t[:,:,-1]))
plt.colorbar()